### Purpose
This notebook is an example on how to do Rossiter-McLaughlin analysis using the Revolutions method (Bourrier+2021). Please read first the transmission_spectroscopy_example to get some understanding on the common set of functions, here they will be only very quickly gone through.


### Setup routine
Same as for transmission spectroscopy.

In [ ]:
import rats.single_use as su

data_directory = '/media/chamaeleontis/Observatory_main/Data_all_raw/WASP_31_b'
main_directory = '/media/chamaeleontis/Observatory_main/Analysis_dataset/WASP-31'
save_directory = main_directory + '/saved_data'
figure_directory = main_directory + '/figures'

su.setup_routine(original_directory= data_directory,
                 main_directory= main_directory,
                 )
os.chdir(main_directory)

### System parameters
Same as for transmission spectroscopy, only added different function for light curve model.

In [ ]:
system_parameters = para.SystemParametersComposite(
    filename= save_directory + '/system_parameters.pkl'
    )
system_parameters.load_NASA_CompositeTable_values(planet_name='WASP-31 b',
                                                  force_load= True)

system_parameters.print_main_values()
system_parameters.model_light_curve()

### Loading and filtering data
Same as for transmission spectroscopy.

In [ ]:
data_raw_A = eso.load_all(main_directory= main_directory,
                          spectra_format= 'CCF_SKYSUB',
                          fiber= 'A',
                          force_skip = False,
                          force_load = False,
                          pkl_name= 'data_raw_CCF_SKYSUB.pkl'
                          )
data_raw_A = sm.get_sublist(data_raw_A, 'Average_S_N', 10, mode='more')
data_raw_A = sm.get_sublist(data_raw_A, 'instrument', 'ESPRESSO', mode='equal')


### Cleaning spectra, sorting, meta dictionary
Same as for transmission spectroscopy

In [ ]:
data_raw_A = sm.sort_spectrum_list(data_raw_A,
                               force_skip = force_skip
                               )
data_raw_A = sm.binning_list(data_raw_A)
system_parameters.spectra_transit_flags(data_raw_A)
system_parameters.calculate_velocities_list(data_raw_A)
system_parameters.add_velocity_system_in_list(data_raw_A)
system_parameters.add_transit_depth_value(data_raw_A)


### Normalize CCF list
Here, the principle behind the function is same, but different function is used. This is because underlying algorithm is different. 

In [ ]:
import rats.analysis.rm as rm
data_normalized = rm.normalize_CCF_list(data_raw_A)

### Calculation of systemic velocity
Systemic velocity can be slightly different between nights, so we measure it on the CCF master out spectra. 

The `rm.calculate_systemic_velocity` fits the master out with Gaussian, and provides the center of it as systemic velocity, including the uncertainty. This velocity is then removed from the data by `rm.remove_systemic_velocity`. `sm.shift_list` cannot be used for small arrays where we shift by non-negligible amount of pixels with respect to the full array (the case for CCF default files).

In [ ]:
import rats.spectra_manipulation as sm

data_SRF = sm.shift_list(data_normalized,
                         shift_BERV=0,
                         shift_v_sys = 0,# No systemic velocity shift
                         shift_v_star = 1,
                         shift_v_planet = 0,
                         )
master_SRF_system = sm.calculate_master_list(data_SRF,
                                          key = 'Transit_partial',
                                          value =False,
                                          )

rm.calculate_systemic_velocity(master_SRF_system, data_SRF)
data_SRF = rm.remove_systemic_velocity(data_SRF)
# Rebin to the same wavelength grid, as the previous function removes this
data_SRF = sm.binning_list(data_SRF)

### Calculation of residual and intrinsic CCFs
Residual and intrinsic CCFs are calculated using the master out spectrum in the rest frame of the star. It is implemented in the `rm.subtract_master_out` function.


In [ ]:
master_SRF_out = sm.calculate_master_list(data_SRF,
                                          key = 'Transit_partial',
                                          value =False,
                                          )
CCF_residual, CCF_intrinsic = rm.subtract_master_out(data_SRF ,master_SRF_out)

### Characterize the quality of the local CCF distribution
First, we need to ensure all the intrinsic CCFs are of good quality. Typically, few needs to be removed from the analysis, particularly at the ingress/egress.

`rm.MCMC_model_local_CCF_list` performs a MCMC fit to each in transit CCF, to get the distribution of the Gaussian profile. Only Gaussian profile is provided for now.

In [ ]:
data_chain_list = rm.MCMC_model_local_CCF_list (CCF_intrinsic)

### Revolutions analysis
Now, after confirming the quality of the data, we can run the full analysis of the dataset using the Revolutions method. To do so, run `rm.MCMC_model_Revolutions`.

In [ ]:
result = rm.MCMC_model_Revolutions(
    system_parameters= system_parameters,
    CCF_intrinsic_list= CCF_intrinsic,
    )

### Final remarks:
This is a short example on how to run the pipeline. More filtering and cleaning steps might be needed for your analysis. Proceed with caution.
